<a href="https://colab.research.google.com/github/greeshmamathad/CStudy.io/blob/main/SE6362phase2-1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from bs4 import BeautifulSoup
import bs4
from html.parser import HTMLParser
from urllib.request import Request,urlopen
from urllib import parse
import re
import numpy as np
from bs4 import SoupStrainer
from datetime import datetime
import pickle
import requests

#### Functions applied in the crawler 

In [ ]:
# the function is used for obtaining all the tags in html file that used to describe the webpage

def drink_soup(link,SoupStrainer):
    html_page = urlopen(link).read()
    soup = BeautifulSoup(html_page,'lxml',parse_only=SoupStrainer)
    return soup

In [ ]:
#the function is used to determine whether the web page is in "html" page

def doctype(soup):
    items = [item for item in soup.contents if isinstance(item, bs4.Doctype)]
    try:
        if items[0] != "html":
            return None
        else:
            return "html"
    except:
        return None

In [ ]:
#some links may have different format but all lead to the same final destination, therefore used this function to obtain the link
#for final destination
#ex.https://www.depaul.edu:443/Pages/default.aspx and http://www.depaul.edu/Pages/default.aspx

def redirect_url(link):
    try: 
        link=urlopen(link).geturl()
    except:
        redirect=requests.get(link)
        link=redirect.url
    else: 
        link
    return link

In [ ]:
#the function is for obtaining unique and normalized link during bfs crawling

def obtain_links(start,soup):
    start=redirect_url(start)
        
    collected_link=[]
    links=soup.findAll('a')
    for link in links:
        attrs=link.attrs
        if 'href' in attrs:
            if attrs['href']=="":
                continue
                
            #normalized the link
            elif attrs['href'][0]=='/':
                l=parse.urljoin(start,attrs['href'])
                if "#" in l:
                    l=l.split("#")[0]

                if "events.depaul.edu/calendar" in l:
                    l=l[:(l.find('calendar')+8)]

                if l not in collected_link:
                    collected_link.append(l)

            #make sure the link is in depaul domain
            elif attrs['href'][0]=='h':
                if "depaul.edu" in attrs['href']:
                    l=attrs['href']
                    
                    if "#" in l:
                        l=l.split("#")[0]

                    if "events.depaul.edu/calendar" in l:
                        l=l[:(l.find('calendar')+8)]

                    if l not in collected_link:
                        collected_link.append(l)           
            else:
                continue
    
    collected_link2=[]
    for i in collected_link:
        new_link=redirect_url(i)
        if "depaul.edu:443" in new_link or "https://sts.is.depaul.edu" in new_link:
            continue
        elif new_link not in collected_link2:
            collected_link2.append(new_link)    
    
    if 'https://www.depaul.edu/Pages/default.aspx' in collected_link2:
        collected_link2.remove('https://www.depaul.edu/Pages/default.aspx')
    
    return collected_link2
            

In [ ]:
#the function that obtains the text of the web page given the html tags of the web page

def obtain_HTML_text(soup):
    text_hold=[]
    
    if "ul" in np.unique([tag.name for tag in soup.find_all()]).tolist():
        menu_tags=soup.find_all("ul")[:2]
        collect=[]
        for tag in menu_tags:
            collect.append(tag.find_all("a"))
        #formating
        clean=[]
        for i2 in range(len(collect)):
            for item in collect[i2]:
                clean.append(item)
        
    for i in np.unique([tag.name for tag in soup.find_all()]).tolist():
        if i not in ["ul",'h6','h5','span','label','li','style', 'script', 'meta', '[document]','div','br','font','figure','footer','form','img','input','nav','tr','video']:
            if i == "a":
                tags=soup.find_all(i,{"rel":None,"class":None,"data-category-slug":None,"data-area":None,"aria-label":None,"style":None})
                try: 
                    no_menu=[]
                    for i3 in set(tags):
                        if i3 not in set(clean):
                            no_menu.append(i3)
                    tags=no_menu

                except:
                    pass
                for t in tags:
                    if 'href' in t.attrs.keys():
                        if "#" in t["href"]:
                            continue
                        if "twitter" in t["href"]:
                            continue
                        if "facebook" in t["href"]:
                            continue
                        if "wordpress.org" in t["href"]:
                            continue
                        if "elegantthemes.com" in t["href"]:
                            continue
                        if t.string != None:
                            text_hold.append(str(t.string))
                            
            else:
                for t2 in set(soup.find_all(i)):
                    if t2.string != None:
                        text_hold.append(str(t2.string))
    return text_hold

#### The crawler function that applies breadth first  crawling strategy

In [ ]:
# The crawler function that applies bfs crawling strategy

def bfs_search(start,max_page):
    
    start=redirect_url(start)
    
    text={}
    
    # the root page
    root_soup=drink_soup(start,None)
    
    #--text--#collect text for the root soup
    text[start]=obtain_HTML_text(root_soup)
    
    #--link--# collect links in the first depth / all the links in the root page
    neighbours_links=obtain_links(start,root_soup)
     
    
    explored_links=[]
    explored_links.append(start)
    print("link#,appended/text_crawled",len(explored_links))

    for node in neighbours_links:
        try:
            if node not in explored_links:
                try:
                    soup=drink_soup(node,None)
                except:
                    try:
                        req = Request(node, headers={'User-Agent': 'Mozilla/5.0'}) #the code prevent block for web scraping
                        soup=drink_soup(req,None)
                    except:
                        print("fail to open:",node)
                        continue
                if len(np.unique(neighbours_links)) < max_page+1000: #in case some page node fail to open
                    #make sure all the pages are html
                    if doctype(soup)=="html":
                        explored_links.append(node)
                        text[node]=obtain_HTML_text(soup)
                        print("link#:",len(explored_links))

                        #collect breadth search link 
                        try: 
                            l_lst=obtain_links(node,soup)
                            for i in l_lst:
                                link=redirect_url(i)
                                if link not in explored_links:
                                    neighbours_links.append(link)
                                    print("Unique link in N",len(np.unique(neighbours_links)))
                        except:
                            explored_links.pop(explored_links.index(node))
                            continue

                else:
                    if len(explored_links) < max_page+50:  # obtain extra data incase some situation happen
                        if doctype(soup)=="html":
                            explored_links.append(node)
                            text[node]=obtain_HTML_text(soup)
                            print("link#:",len(explored_links))
                    else:
                        break

            #early stoping criteria 2, see if we scrap enough page
            elif len(explored_links) > max_page+50:
                break
                
        except:
            print("last node:",node)
            return text,neighbours_links,explored_links
            
            

    return text

#### Crawling 5000 web pages

In [ ]:
start="https://www.depaul.edu"
max_page=10

In [ ]:
#it will take about 5 hours to crawl 
start_time = datetime.now()

save_dict=bfs_search(start,max_page)

end_time = datetime.now()
print('Duration: {}'.format(end_time - start_time))

link#,appended/text_crawled 1
link#: 2
Unique link in N 77
Unique link in N 77
Unique link in N 77
Unique link in N 78
Unique link in N 78
Unique link in N 78
Unique link in N 78
Unique link in N 78
Unique link in N 78
Unique link in N 78
Unique link in N 78
Unique link in N 78
Unique link in N 78
Unique link in N 78
Unique link in N 79
Unique link in N 80
Unique link in N 81
Unique link in N 81
Unique link in N 82
Unique link in N 82
Unique link in N 82
Unique link in N 82
Unique link in N 82
Unique link in N 82
Unique link in N 82
Unique link in N 82
Unique link in N 82
Unique link in N 82
link#: 3
Unique link in N 82
Unique link in N 82
Unique link in N 83
Unique link in N 83
Unique link in N 83
Unique link in N 83
Unique link in N 83
Unique link in N 83
Unique link in N 83
Unique link in N 83
Unique link in N 83
Unique link in N 83
Unique link in N 83
Unique link in N 84
Unique link in N 85
Unique link in N 86
Unique link in N 87
Unique link in N 88
Unique link in N 89
Unique link 

In [ ]:
len(save_dict)

#### Save the crawled web pages into pickle file

In [ ]:
#with open('5000_links_texts.pickle', 'wb') as handle:
    #pickle.dump(save_dict, handle)#,protocol=pickle.HIGHEST_PROTOCOL)